<a href="https://colab.research.google.com/github/vktiwary007/stock-move-eureka/blob/main/stockMovement2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.4.1
Eager mode:  True
Hub version:  0.11.0
GPU is NOT AVAILABLE


In [2]:
import io
#df = pd.read_csv('Apple60.csv', skiprows=0)

from sklearn.model_selection import train_test_split
data = pd.read_csv('Apple60.csv', sep=',', header=0)
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [23]:
data[:10]

,Source,News,Movement
0,www.businesswire.com,NAPE Summit Week will also feature the annual ...,1
1,seekingalpha.com,Bearish Calls Lumentum (NASDAQ: LITE ): It's a...,0
2,seekingalpha.com,The areas to be concerned are that there are f...,1
3,seekingalpha.com,Amazon will soon resume selling the Apple TV (...,1
4,seekingalpha.com,"With such a large revenue pie, naturally we be...",0
5,seekingalpha.com,"Facing questions from tech analysts, Apple (NA...",0
6,www.thestreet.com,Cramer and the AAP team say investors need to ...,0
7,seekingalpha.com,One day after Apple acknowledged that it has b...,0
8,www.thestreet.com,1 . -- U.S. stock futures turned lower on Tues...,0
9,seekingalpha.com,"Meanwhile, eight more lawsuits have been filed...",0


In [3]:
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from tqdm import tqdm
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

MAX_NB_WORDS = 1000
max_seq_len = 100

raw_docs_train = train_df['News'].tolist()
raw_docs_test = test_df['News'].tolist()

num_classes = 2
processed_docs_train = []

stop_words = set(stopwords.words('english'))

for doc in tqdm(raw_docs_train):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_docs_train.append(" ".join(filtered))
processed_docs_test = []

for doc in tqdm(raw_docs_test):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_docs_test.append(" ".join(filtered))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  

word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index

word_seq_train = pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = pad_sequences(word_seq_test, maxlen=max_seq_len)

  0%|          | 79/31801 [00:00<00:45, 691.84it/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


100%|██████████| 7951/7951 [00:09<00:00, 839.31it/s]


In [38]:
import requests, zipfile, io
zip_file_url = "https://nlp.stanford.edu/data/glove.840B.300d.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

print("GloVE available locally")

GloVE available locally


In [5]:
import codecs

embeddings_index = {}
#f = codecs.open(‘crawl-300d-2M.vec’, encoding=’utf-8')
# for Glove
f = codecs.open('glove.840B.300d.txt', encoding='utf-8')
for line in tqdm(f):
  values = line.rstrip().rsplit(' ')
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

2196018it [04:31, 8074.50it/s]


In [12]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.11821563, -0.38141903, -0.00576665,  0.4024705 , -0.4506181 ,
        -0.49942222,  0.7247262 ,  0.37873062,  0.67162323,  0.1313145 ,
         0.21069519,  0.37749547,  0.39727092, -0.77833736,  0.8862999 ,
        -0.66263425, -0.40579197,  0.14556947,  0.1540871 ,  0.57739764,
        -0.39237094, -0.39210856,  0.40515414,  0.2835647 ,  0.71169597,
         0.28195596, -0.39363083,  0.19764623, -0.28529963, -0.02440625,
         0.07258959,  0.00366513,  0.74614835, -0.4447853 , -0.18952131,
         0.1531296 ,  0.37783742, -0.60432875, -0.13870297, -0.42431712,
        -0.6368046 , -0.23387073,  0.10721027,  0.60974413, -0.10874098,
        -0.84399396,  0.13302684,  0.29855415,  0.3308882 ,  0.18879387],
       [ 0.1564894 , -0.23767409, -0.26291662,  0.18460633, -0.41442817,
        -0.5455462 , -0.10182314,  0.4068529 ,  0.23047605,  0.3656314 ,
        -0.12204684,  0.34686607,  0.6008024 , -0.31858292,  0.30129915,
 

In [6]:
words_not_found = []
embed_dim = 300
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
  if i >= nb_words:
     continue
  embedding_vector = embeddings_index.get(word)
  
  if (embedding_vector is not None) and len(embedding_vector) > 0:
     embedding_matrix[i] = embedding_vector
  else:
     words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

number of null word embeddings: 18


In [9]:
from keras.layers import BatchNormalization
from keras.layers import Embedding, Bidirectional, Dense, Dropout, LSTM, Flatten, Conv1D, MaxPooling1D, Activation
from keras.initializers import Constant

model = tf.keras.Sequential()
model.add(Embedding(nb_words, embed_dim, input_length=max_seq_len, weights=[embedding_matrix],trainable=False))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(2, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          300000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 96, 64)            96064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 24, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 462,266
Trainable params: 162,266
Non-trainable params: 300,000
__________________________________________

In [14]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical

labels = to_categorical(np.asarray(train_df['Movement']))
y_train = np.array(train_df['Movement'])

print(labels.shape)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es_callback = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(word_seq_train, np.array(labels), epochs=10, validation_split=0.3)

(31801, 2)
Epoch 1/10
895/895 [==============================] - 70s 75ms/step - loss: 0.6951 - accuracy: 0.5085 - val_loss: 0.6926 - val_accuracy: 0.5086
Epoch 2/10
895/895 [==============================] - 67s 75ms/step - loss: 0.6906 - accuracy: 0.5316 - val_loss: 0.6947 - val_accuracy: 0.5017
Epoch 3/10
895/895 [==============================] - 67s 75ms/step - loss: 0.6839 - accuracy: 0.5529 - val_loss: 0.6967 - val_accuracy: 0.5115
Epoch 4/10
895/895 [==============================] - 68s 76ms/step - loss: 0.6664 - accuracy: 0.5818 - val_loss: 0.7030 - val_accuracy: 0.5187
Epoch 5/10
895/895 [==============================] - 68s 76ms/step - loss: 0.6389 - accuracy: 0.6169 - val_loss: 0.7296 - val_accuracy: 0.5124
Epoch 6/10
895/895 [==============================] - 68s 76ms/step - loss: 0.6044 - accuracy: 0.6531 - val_loss: 0.7475 - val_accuracy: 0.5247
Epoch 7/10
895/895 [==============================] - 67s 75ms/step - loss: 0.5776 - accuracy: 0.6627 - val_loss: 0.7918 - va

In [17]:
df_test = pd.read_csv('Apple60_Major.csv', skiprows=0)
#print(df_test.iloc[:, -1].values)
df_test["Movement"].replace({-1: 0}, inplace=True)
#test_data = np.array(hub_layer(df_test["News"]))

#model.predict(word_seq_test)


array([[0.86559093, 0.14475647],
       [0.15704784, 0.8414612 ],
       [0.4064761 , 0.5890338 ],
       ...,
       [0.6626729 , 0.29772514],
       [0.5685262 , 0.44615674],
       [0.97659117, 0.02298823]], dtype=float32)

In [14]:
classifier.fit(hub_layer(train_examples), y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [15]:
test_val = np.array(hub_layer(test_examples))

print(test_val[:10])

[[ 7.12334439e-02 -2.19031528e-01 -3.48590881e-01  6.60696998e-02
  -2.29958817e-01  7.51711577e-02  2.24154130e-01 -7.20842257e-02
   2.02052578e-01 -1.01056278e-01  7.60269165e-02  3.42676282e-01
   1.45717617e-02 -1.11079566e-01  2.40788341e-01  3.22015166e-01
   3.21642533e-02  4.52669293e-01 -4.55906130e-02  7.85075128e-02
  -2.53289223e-01 -3.95622551e-02  7.34008774e-02 -1.27158761e-01
  -3.28406431e-02 -1.25006929e-01 -1.13356918e-01 -1.37009054e-01
  -1.14342615e-01  1.74757615e-01  4.26033974e-01  2.72094667e-01
   2.08017919e-02 -1.73620842e-02 -9.12332609e-02 -5.83611280e-02
   2.32044682e-01 -1.27907902e-01 -1.25517219e-01  1.08377382e-01
   1.78981408e-01 -1.01789393e-01  1.29062980e-01  6.44956753e-02
   3.99186090e-02 -7.44161531e-02 -2.62955695e-01  1.67494416e-01
   3.06270093e-01  6.16518974e-01]
 [ 5.83040491e-02  1.55362993e-01 -8.57881233e-02 -1.51026607e-01
  -2.49560565e-01 -1.71633616e-01  6.98898435e-01  3.78702611e-01
  -4.60917428e-02  1.72753289e-01 -1.9155

In [16]:
y_pred = classifier.predict(test_val)

In [17]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
print(accuracy_score(y_test, y_pred)*100,"%")
print(confusion_matrix(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))

53.100238963652366 %
[[2247 1812]
 [1917 1975]]
0.5143898945175152
0.5215209928703459
0.5074511819116135


In [18]:
df_test = pd.read_csv('Apple60_Major.csv', skiprows=0)
#print(df_test.iloc[:, -1].values)
df_test["Movement"].replace({-1: 0}, inplace=True)
test_data = np.array(hub_layer(df_test["News"]))

print(test_data[:10])

[[ 3.13545793e-01 -1.23975776e-01 -1.70429930e-01 -3.49904187e-02
  -3.58242057e-02 -1.88798577e-01  4.39916462e-01 -1.29881210e-03
   6.71013445e-02  1.16763636e-02 -3.14179473e-02  1.77670971e-01
   1.02066256e-01  4.23686802e-02  1.51786089e-01 -1.92925498e-01
  -4.15387869e-01  1.72586322e-01  2.56283581e-01 -3.39321703e-01
   4.59879497e-03 -2.07928702e-01  1.31040663e-01  8.24995786e-02
  -1.46420583e-01  9.68253911e-02 -1.95102215e-01  2.11510524e-01
   1.45782251e-02 -9.08587575e-02  1.23649403e-01 -3.54639590e-02
   4.34994429e-01 -2.83502400e-01 -4.69967425e-02  1.43503875e-01
   1.37878478e-01 -1.77231729e-01  7.52000213e-02 -6.56884611e-02
  -2.13369891e-01  4.16275896e-02  2.60824144e-01  2.92564899e-01
  -1.34889305e-01 -3.78095359e-01 -6.74926639e-02 -6.89061009e-04
  -2.41003126e-01 -8.17034766e-02]
 [ 1.58091709e-01  8.09366778e-02 -1.00125924e-01  3.08011591e-01
  -1.21869713e-01 -2.66389757e-01  2.65746415e-01  1.15882643e-01
   2.80758470e-01  8.64161253e-02 -1.0979

In [19]:
y_testSample = df_test.iloc[:, -1].values

print(y_testSample[:10])

[0 0 0 0 0 0 0 0 0 0]


In [20]:
y_predTest = classifier.predict(test_data)

In [21]:
print(accuracy_score(y_testSample, y_predTest)*100,"%")
print(confusion_matrix(y_testSample, y_predTest))
print(f1_score(y_testSample, y_predTest))
print(precision_score(y_testSample, y_predTest))
print(recall_score(y_testSample, y_predTest))

50.91420534458509 %
[[178 162]
 [187 184]]
0.5132496513249651
0.5317919075144508
0.49595687331536387
